# Validata Delay calibration

In [25]:
from ska_sdp_instrumental_calibration.workflow.stages import (
    load_data_stage,
    predict_vis_stage,
    
)
from ska_sdp_datamodels.visibility.vis_io_ms import export_visibility_to_ms
from ska_sdp_instrumental_calibration.scheduler import UpstreamOutput
import matplotlib.pyplot as plt
import os
%matplotlib inline

### Data Generation

We had simulated data using Oskar. The simulation scripts are present in `scripts/ska_low_sim`. (refer [confluence page](https://confluence.skatelescope.org/display/SE/DHR-311%3A+Script+to+simulate+SKA-LOW+visibilities))

We had simulated two measurement sets. One only with delay effects and second with bandpass and delay effects.

#### Simulation Configuration for only Delay

For simulation following configuration is used. (further refered as `delay_sim.yaml`)

``` yaml
scenario: "low40s-model"          # Scenario name (used for output folder prefix)

# ===============================
# Global simulation parameters
# ===============================

n_stations: 40                                         # Number of stations
tel_model: "./telescope-models/SKA-Low_AA2_40S_rigid-rotation_model.tm" # Telescope model directory

simulation_start_frequency_hz: 123.0e6                  # Start frequency (Hz)
simulation_end_frequency_hz: 153.0e6                    # End frequency (Hz)
correlated_channel_bandwidth_hz: 21.70138888888889e3    # Channel width (Hz)

observing_time_mins: 10                              # Observation duration (minutes)
sampling_time_sec: 3.3973862400000003                   # Dump/integration time (seconds)

fields:
  EoR2:
    Cal1:
      ra_deg: 197.914612
      dec_deg: -22.277973
      scan_id_start: 300
      transit_time: "2000-01-03 22:33:30.000"

# ==================================
# Options for generate_gaintable.py
# ==================================

generate_gaintable:
  output_gaintable: &gen_gaintable "./gaintables/custom_gaintable.h5"

  station_offset: true              # Apply per-station amplitude/phase offsets
  time_variant: true                # Apply time-dependent effects

  rfi: false                        # Inject RFI band
  rfi_start_freq_hz: 154.25347222228538e6        # Hz
  rfi_end_freq_hz: 159.8090277778474e6           # Hz

  plot: true                        # Generate diagnostic plots
  plot_output_dir: "./gaintables/generation_plots/"

# ===============================
# Options for run_sim.py
# ===============================

run_sim:
  oskar_sif: "./OSKAR-2.11.1-Python3.sif" # Path to OSKAR Singularity image

  # GLEAM sky model. Optional. Comment to disable.
  gleam_file: "./sky-models/GLEAM_EGC.fits" # GLEAM catalogue FITS file
  field_radius_deg: 10.0            # Radius of field of view (degrees)

  # Corruptions to be applied. All are optional. Comment to disable.
  # gaintable: *gen_gaintable           # Gaintable containing bandpass corruptions
  cable_delay: "./cable_delays/cable_length_error_40s.txt" # Cable delay error file
  # tec_screen: "./tec/calibrator_iono_tec.fits" # Ionospheric TEC screen FITS

  # Imaging parameters using wsclean. Optional. Comment to disable.
  create_dirty_image: true          # Whether to run wsclean imaging
  image_size: 1024                  # Image size (pixels)
  pixel_size: "2arcsec"             # Pixel size (angular units)

  # Extra parameters to pass directly to run_oskar.py
  run_oskar_extra_params: "--use-gpus --double-precision"
```

Follow steps mentioned in confluence page for data and enviornment setup. Run the following command to simulate visibilities. 
`python run_sim.py delay_sim.yaml`

#### Simulation Configuration for Delay and bandpass

For simulation following configuration is used. (further refered as `bandpass_delay_sim.yaml`)

``` yaml
scenario: "low40s-model"          # Scenario name (used for output folder prefix)

# ===============================
# Global simulation parameters
# ===============================

n_stations: 40                                         # Number of stations
tel_model: "./telescope-models/SKA-Low_AA2_40S_rigid-rotation_model.tm" # Telescope model directory

simulation_start_frequency_hz: 123.0e6                  # Start frequency (Hz)
simulation_end_frequency_hz: 153.0e6                    # End frequency (Hz)
correlated_channel_bandwidth_hz: 21.70138888888889e3    # Channel width (Hz)

observing_time_mins: 10                              # Observation duration (minutes)
sampling_time_sec: 3.3973862400000003                   # Dump/integration time (seconds)

fields:
  EoR2:
    Cal1:
      ra_deg: 197.914612
      dec_deg: -22.277973
      scan_id_start: 300
      transit_time: "2000-01-03 22:33:30.000"

# ==================================
# Options for generate_gaintable.py
# ==================================

generate_gaintable:
  output_gaintable: &gen_gaintable "./gaintables/custom_gaintable.h5"

  station_offset: true              # Apply per-station amplitude/phase offsets
  time_variant: true                # Apply time-dependent effects

  rfi: false                        # Inject RFI band
  rfi_start_freq_hz: 154.25347222228538e6        # Hz
  rfi_end_freq_hz: 159.8090277778474e6           # Hz

  plot: true                        # Generate diagnostic plots
  plot_output_dir: "./gaintables/generation_plots/"

# ===============================
# Options for run_sim.py
# ===============================

run_sim:
  oskar_sif: "./OSKAR-2.11.1-Python3.sif" # Path to OSKAR Singularity image

  # GLEAM sky model. Optional. Comment to disable.
  gleam_file: "./sky-models/GLEAM_EGC.fits" # GLEAM catalogue FITS file
  field_radius_deg: 10.0            # Radius of field of view (degrees)

  # Corruptions to be applied. All are optional. Comment to disable.
  gaintable: *gen_gaintable           # Gaintable containing bandpass corruptions
  cable_delay: "./cable_delays/cable_length_error_40s.txt" # Cable delay error file
  # tec_screen: "./tec/calibrator_iono_tec.fits" # Ionospheric TEC screen FITS

  # Imaging parameters using wsclean. Optional. Comment to disable.
  create_dirty_image: true          # Whether to run wsclean imaging
  image_size: 1024                  # Image size (pixels)
  pixel_size: "2arcsec"             # Pixel size (angular units)

  # Extra parameters to pass directly to run_oskar.py
  run_oskar_extra_params: "--use-gpus --double-precision"
```

First generate the gaintable using the command,

```
python generate_gaintable.py bandpass_delay_sim.yaml
```

Run the simulation.
```
python run_sim.py delay_sim.yaml
```

## Helper functions

In [31]:
from numpy import float64, array, loadtxt

c = 299792458.0

def get_cable_delays_in_ns(cable_delays_path):
    delays = loadtxt(cable_delays_path, dtype=float64)
    delays =  array(list(map(lambda delay: (float64(delay) / c) * 1e9, delays)))
    return delays


# get_cable_delays_in_ns("/home/nitin/Work/ska/data-simulation/ska_low_sim/cable_delays/cable_length_error.txt")

In [32]:
def plot_delays_in_ns(delays, cable_delays, ref_ant, path_prefix):
    plt.style.use('default') 
    fig = plt.figure(layout="constrained", figsize=(10, 5))
    fig.suptitle("Delays in ns", fontsize=16)
    actual_delay_plt, expected_delay_plt = fig.subplots(1, 2)

    actual_delay_plt.set_title("Actual Delays")
    actual_delay_plt.set_xlabel("Antenna")
    actual_delay_plt.set_ylabel("Delays (ns)")

    expected_delay_plt.set_title("Expected Delays")
    expected_delay_plt.set_xlabel("Antenna")
    expected_delay_plt.set_ylabel("Delays (ns)")

    actual_delay_plt.plot(abs(delays))
    expected_delay_plt.plot(abs(cable_delays))

    # handles, labels = sub_plt.get_legend_handles_labels()
    # fig.legend(handles, labels, title="Baselines", loc="outside center right")
    fig.savefig(f"{path_prefix}/delays-with-ref_ant-{ref_ant}.png")
    plt.close(fig)

## Pipeline Setup

In [28]:
cache = "../../cache"

In [29]:
config = f"""
parameters:
  load_data:
    nchannels_per_chunk: 32
    ntimes_per_ms_chunk: 5
    cache_directory: {cache}
    ack: false
    datacolumn: DATA
    field_id: 0
    data_desc_id: 0
  predict_vis:
    beam_type: everybeam
    normalise_at_beam_centre: true
    eb_ms: null
    eb_coeffs: /home/ska/Work/data/INST/sim/coeffs 
    gleamfile: null #/home/ska/Work/data/INST/sim/gleamegc.dat
    lsm_csv_path: /home/ska/Work/data/INST/lg3/sky_model_cal.csv
    fov: 10.0
    flux_limit: 1.0
    alpha0: -0.78
  bandpass_calibration:
    run_solver_config:
      solver: gain_substitution
      refant: 0
      niter: 10
      phase_only: false
      tol: 1.0e-06
      crosspol: false
      normalise_gains: null
      timeslice: null
    plot_config:
      plot_table: true
      fixed_axis: true
    visibility_key: vis
    export_gaintable: true
  delay_calibration:
    oversample: 16
    plot_config:
      plot_table: true
      fixed_axis: true
      anotate_stations: true
    export_gaintable: true
pipeline:
  load_data: true
  predict_vis: true
  bandpass_calibration: true
  flag_gain: false
  generate_channel_rm: false
  delay_calibration: true
  smooth_gain_solution: false
  export_visibilities: false
  export_gain_table: false
global_parameters:
  experimental:
    pipeline: []
"""

In [30]:
artifacts_prefix_path = "./delay_validation_artefacts"
config_path = artifacts_prefix_path + "/config.yml"

os.makedirs(artifacts_prefix_path, exist_ok=True)

with open(config_path, "w") as f:
    f.write(config)

## With Delay Effects

Running the pipeline with simulated data with only delay effects.

In [41]:
input_data = "/home/ska/Work/data-simulation/with_delay_effects/low40s-model-081025_132513/visibility.scan-300.ms"
delay_artefacts_path = artifacts_prefix_path + "/delay"
os.makedirs(delay_artefacts_path, exist_ok=True)

In [ ]:

## Running the pipeline
!time ska-sdp-instrumental-calibration run --input $input_data --config $config_path --dask-scheduler "tcp://127.0.0.1:34555" --output $delay_artefacts_path

### Comparision

After the pipeline is finished, we need to compare delays from pipeline and delays used in data simulation.

Data simluation uses cable delay text file which contains delays in meters. We need to convert the delays in meters to neno seconds. We have created a function `get_cable_delays_in_ns` that loads the data in text file, converts the delays into neno seconds.

`get_cable_delays_in_ns` uses formula.

$$
\mathrm{delay}_{\mathrm{ns}} = \frac{\mathrm{length}_{\mathrm{m}}}{c} \times 10^9
$$

where $c = 299792458 m/s$

As the delays from the pipeline have `time`, `antenna` and `pol` as dimensions, we converse it to result in just one dimension as `antenna`. Next we take delay with reference to an antenna and plot them. The plot will be created for the reference antenna. To create another plot, provide another reference antenna.

**Verfication**
1. Plot for delays should match.
2. Phase in plots of gaintable after applying delayed effect, should be zero.

In [42]:
## plot delays

import h5py
import numpy as np

delay_clock_file = "/home/nitin/Work/ska/sdp/ska-sdp-instrumental-calibration/notebooks/validations/delay_validation_artefacts/ska_sdp_instrumental_calibration_2025-10-09T11:19:19/delay.clock.h5parm"

cable_delays_path = "/home/nitin/Work/ska/data-simulation/ska_low_sim/cable_delays/cable_length_error.txt"

cable_delays = get_cable_delays_in_ns(cable_delays_path)
ref_ant = 0
with h5py.File(delay_clock_file) as f:
    delays = f["sol000"]["clock000"]["val"][()][0]
    delays = delays.mean(axis=1)
    plot_delays_in_ns(delays - delays[ref_ant], cable_delays - cable_delays[ref_ant], ref_ant, delay_artefacts_path)
   

## With bandpass and Delay

In [44]:
input_data = "/home/ska/Work/data-simulation/with_delay_effects/low40s-model-081025_132513/visibility.scan-300.ms"
delay_artefacts_path = artifacts_prefix_path + "/bandpass-delay"
os.makedirs(delay_artefacts_path, exist_ok=True)

In [ ]:

## Running the pipeline
!time ska-sdp-instrumental-calibration run --input $input_data --config $config_path --dask-scheduler "tcp://127.0.0.1:34555" --output $delay_artefacts_path

### Comparision

In [ ]:
## plot delays

import h5py
import numpy as np

delay_clock_file = "/home/nitin/Work/ska/sdp/ska-sdp-instrumental-calibration/notebooks/validations/delay_validation_artefacts/ska_sdp_instrumental_calibration_2025-10-09T11:19:19/delay.clock.h5parm"

cable_delays_path = "/home/nitin/Work/ska/data-simulation/ska_low_sim/cable_delays/cable_length_error.txt"

cable_delays = get_cable_delays_in_ns(cable_delays_path)
ref_ant = 39
with h5py.File(delay_clock_file) as f:
    delays = f["sol000"]["clock000"]["val"][()][0]
    delays = delays.mean(axis=1)
    plot_delays_in_ns(delays - delays[ref_ant], cable_delays - cable_delays[ref_ant], ref_ant, delay_artefacts_path)
   